In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined b
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#y the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [78]:
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [86]:
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected=True)
#import plotly.express as px


In [87]:
train["date"] = pd.to_datetime(train.date)
transactions.date = pd.to_datetime(transactions.date)

In [88]:
sales_txn = pd.merge(train.groupby(["date","store_nbr"]).sales.sum().reset_index(),
                     transactions, how = "left") 
sales_txn

# Transactions
Transactions show the number of invoice generated at the store for the respective date.
It can be clearly seen in the graphs below that - 
1. Transactions is highly correlated with sales.
2. The number of transactions increases every year in Decembor month.

In [89]:
print("Spearman Correlation between Total Sales and Transactions : {:,.4f} " 
          .format(sales_txn.corr('spearman').sales.loc["transactions"]))
sns.scatterplot(x = "transactions",y = "sales", data = sales_txn , hue = 'store_nbr')
px.line(transactions.sort_values(["store_nbr","date"]),x = "date", 
        y = "transactions" , color = "store_nbr", title = "Transactions")

In [90]:
#Confirming the increase in sales in December using a boxplot
temp = transactions.copy()
temp["Year"] = temp.date.dt.year
temp["Month"] = temp.date.dt.month
px.box(temp, x = "Year" , y = "transactions" , color = "Month", title = "Transactions")

# Monthly Average Transactions
As sales increases at the end of year, the transactions also increases after the spring and then falls down.


In [84]:
a = transactions.set_index("date").resample("M").transactions.mean().reset_index()
a["Year"] = a.date.dt.year
px.line(a,x = 'date',y = 'transactions', color = 'Year')


# Transaction Analysis based on Day of week

In [85]:
a = transactions.copy()
a["dayofweek"] = a.date.dt.dayofweek + 1
a['Year'] = a.date.dt.year
a = a.groupby(['Year','dayofweek']).transactions.mean().reset_index()
px.line(a,x = 'dayofweek', y = "transactions", color = 'Year')